In [327]:
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

---

## 1. Price parsing

- `original_price`: Giá gốc (giá chưa giảm, đơn vị: VND)
- `sale_price`: Giá khuyến mãi (giá đã giảm, đơn vị: VND)
- `precent_discount`: Phần trăm số tiền đã giảm (đơn vị: %)

<pre>
- Có giá khuyến mãi: 
    &lt;fare-sale&gt; chứa giá khuyến mãi
        &lt;fareSmall&gt; chứa giá gốc và phần trăm ưu đãi
- Không có giá khuyến mãi:
    &lt;fare&gt; chứa giá gốc
        &lt;fareSmall&gt; không chứa dữ liệu
</pre>

In [328]:
def has_no_discount_price(block) -> bool:
    '''
    Phân loại chuyến xe này có giá khuyến mãi hay không \n
    True: không có giá khuyến mãi \n
    False: có giá khuyến mãi
    '''
    fare = block.find('div', class_='fare')      
    
    if fare: 
        return True  # khong co gia khuyen mai
    
    return False     # co gia khuyen mai

def parse_fare(block):
    '''
    Trích xuất dữ liệu về giá của chuyến xe khi không có giá khuyến mãi \n
    Trả về: giá gốc, giá khuyến mãi = None, phần trăm khuyến mãi = None
    '''

    sale_price = None
    percent_discount = None

    fare = block.find('div', class_='fare')
    original_price = fare.get_text(strip=True).replace("đ", "").replace('Từ ', '').strip()    
    
    return {
        "original_price": original_price ,
        "sale_price":sale_price,
        "percent_discount": percent_discount
        }

def parse_fare_small(block):
    '''
    Dành cho trường hợp có khuyến mãi \n
    Trả về: giá gốc, phần trăm khuyến mãi
    '''

    fare_small = block.find('div', class_='fareSmall')

    original_price = fare_small.find('div', class_='small').get_text(strip=True).replace("đ", "").strip() if fare_small else None
    try:
        if fare_small.find('div', class_='percent'):
            percent_discount = fare_small.find('div', class_='percent').get_text(strip=True)
        else:
            percent_discount = None
    except Exception:
        percent_discount = None
        
    return original_price, percent_discount

def parse_sale_price_info(block):
    '''
    Trích xuất dữ liệu về giá của chuyến xe khi có giá khuyến mãi \n
    Trả về: giá gốc, giá khuyến mãi, phần trăm khuyến mãi
    '''
    
    fare_sale = block.find('div', class_='fare-sale')
    original_price, percent_discount = parse_fare_small(block)
    sale_price = None
    if fare_sale and fare_sale.get_text(strip=True):
        sale_price = fare_sale.get_text(strip=True).replace("đ", "").replace('Từ ', '').strip()
        
    return {
        "original_price": original_price ,
        "sale_price":sale_price,
        "percent_discount": percent_discount
        }

In [329]:
def parse_price(block):
    '''
    Trích xuất dữ liệu về giá của chuyến xe \n
    Trả về: giá gốc, giá khuyến mãi (nếu có), phần trăm khuyến mãi (nếu có)
    '''
    
    if has_no_discount_price(block):
        return parse_fare(block)
    else:
        return parse_sale_price_info(block)

---

## 2. Bus info parsing

In [330]:
def parser_trip_bus_info(container):
    '''
    Trích xuất thông tin từ một container chứa thông tin chuyến đi. \n
    Trả về Tuple: tên nhà xe, đánh giá nhà xe, loại ghế.
    '''
    
    # bus name / company name
    bus_element = container.find('div', class_='bus-name')
    bus_name = bus_element.get_text(strip=True) if bus_element else None

    # bus rating
    rating_element = container.find('div', class_='bus-rating').find('span')
    bus_rating = rating_element.get_text(strip=True) if rating_element else None

    # seat_type
    seat_type = container.find('div', class_='seat-type')
    seat_type = seat_type.get_text(strip=True) if seat_type else None

    return {
        'bus_name': bus_name,
        'bus_rating': bus_rating,
        'seat_type': seat_type
    }

---

## 3. Route parsing

In [331]:
# Dữ liệu này nằm ở ô filter chuyến đi

def parse_route_info(block):
    '''
    Trích xuất dữ liệu từ filter của trang web \n
    Trả về: ngày khởi hành, nơi xuất phát (thành phố hiện tại), nơi đến (nơi đặt vé đến)
    '''

    departure_date, start_point, destination = None, None, None

    try:
        departure_date = block.find('p', class_='date-input-value').get_text(strip=True)
        start_point = block.find(id="from_input").get('value')
        destination = block.find(id="to_input").get('value')
    except Exception:
        pass

    return {
        'departure_date': departure_date,
        'start_point': start_point,
        'destination': destination
    }

---

## 4. Details trip info

### 4.1 Departure

In [332]:
# Dữ liệu này nằm trong container > 'from_content'

def parse_departure_trip_info(from_content):
    """
    Trích xuất thông tin điểm đi từ một container 'from_content'.
    Trả về một tuple chứa: giờ khởi hành, địa điểm đón khách.
    """
    # nếu container không tồn tại, trả về giá trị None cho tất cả
    if not from_content:
        return None, None

    # departure time
    departure_time_element = from_content.find('div', class_='hour')
    departure_time = departure_time_element.get_text(strip=True) if departure_time_element else None

    # departure place
    from_place_tag = from_content.find('div', class_='place')
    pick_up_point = from_place_tag.get_text(strip=True) if from_place_tag else None
    
    return {
        'departure_time': departure_time,
        'pick_up_point': pick_up_point
    }

### 4.2 Arrival

In [333]:
def parse_arrival_trip_info(to_content):
    """
    Trích xuất thông tin điểm đến từ một container 'to_content'.\n
    Trả về một tuple chứa: ngày đến, thời gian đến, điểm trả khách.
    """
    
    # nếu container không tồn tại, trả về giá trị None cho tất cả
    if not to_content:
        return None, None, None

    # lấy ngày đến
    date_arrival_tag = to_content.find('span', class_="text-date-arrival-time")
    arrival_date = date_arrival_tag.get_text(strip=True) if date_arrival_tag else None
    
    
    # lấy giờ và địa điểm trả khách
    content_to_info = to_content.find('div', class_='content-to-info')
    if content_to_info:
        to_hour_tag = content_to_info.find('div', class_='hour')
        arrival_time = to_hour_tag.get_text(strip=True) if to_hour_tag else None
        
        dropoff_place_element = content_to_info.find('div', class_='place')
        drop_off_point = dropoff_place_element.get_text(strip=True) if dropoff_place_element else None
        
    return {
        'arrival_date': arrival_date,
        'arrival_time': arrival_time,
        'drop_of_point': drop_off_point
    }


---

In [334]:
def parse_trip_timing(container):
    """
    Trích xuất thông tin chi tiết về chuyến đi (giờ, nơi đi - đến, thời gian di chuyển). \n
    Trả về: dict chứa thông tin khởi hành, điểm đến và thời lượng chuyến.
    """
    
    # Tìm khối chứa thông tin đi và đến
    from_to_content = container.find('div', class_="from-to-content")

    # Nếu không tìm thấy, trả về dict rỗng có cấu trúc sẵn
    if not from_to_content:
        return {
            "duration": None,
            "from_hour": None,
            "from_place": None,
            "departure_date": None,
            "arrival_date": None,
            "to_hour": None,
            "to_place": None,
        }

    # Lấy thông tin nơi khởi hành
    from_content = from_to_content.find('div', class_='content from')
    dict_departure_info = parse_departure_trip_info(from_content)

    # Lấy thông tin nơi đến
    to_content = from_to_content.find('div', class_='content to')
    dict_arrival_info = parse_arrival_trip_info(to_content)

    # Lấy thời gian di chuyển
    duration_tag = from_to_content.find('div', class_="duration")
    duration = duration_tag.get_text(strip=True) if duration_tag else None

    # Gộp toàn bộ thông tin lại
    trip_data = dict_departure_info | dict_arrival_info | {'duration': duration}
    
    return trip_data


In [335]:
def compile_trip_info(block):
    '''
    Tập hợp toàn bộ thông tin của 1 chuyến xe từ 1 khối dữ liệu (block). \n
    Trả về: dict chứa thông tin xe, lịch trình và giá vé.
    '''
    
    # Lấy thông tin chính của nhà xe
    dict_bus_info = parser_trip_bus_info(block)
    
    # Lấy thông tin giờ đi - giờ đến, điểm đón - trả
    dict_trip_details = parse_trip_timing(block)
    
    # Lấy thông tin giá vé (giá gốc, giá khuyến mãi)
    dict_price = parse_price(block)
    
    # Gộp tất cả dữ liệu vào 1 dictionary duy nhất
    trip_data = dict_bus_info | dict_trip_details | dict_price
    
    return trip_data


---

## Parsing rating

In [336]:
def parse_ratings_from_container(container):
    '''
    Trích xuất thông tin đánh giá (rating) của từng nhà xe trong 1 container. \n
    Trả về: list các cặp (rate_title, rate_point) hoặc [(None, None)] nếu không có dữ liệu.
    '''
    try:
        ratings = []
        rate_divs = container.find_all('div', class_='rate-title')   # Tìm tất cả khối chứa thông tin đánh giá
        
        for rate_div in rate_divs:
            rate_ps = rate_div.find_all('p')   # Mỗi phần tử chứa tiêu đề và điểm
            if len(rate_ps) >= 2:
                rate_title = rate_ps[0].get_text(strip=True)   # Tiêu đề đánh giá
                rate_point = rate_ps[1].get_text(strip=True)   # Điểm đánh giá
                ratings.append((rate_title, rate_point))       # Lưu vào danh sách
        
        if ratings:
            return ratings     # Trả về danh sách nếu có dữ liệu
        else:
            return [(None, None)]   # Không có dữ liệu đánh giá
            
    except Exception:
        return [(None, None)]   # Trường hợp lỗi vẫn trả về giá trị mặc định


total

In [337]:
# Gộp dữ liệu chuyến xe và các đánh giá lại với nhau
def extract_all_trips(soup):
    '''
    Trích xuất và gộp thông tin chuyến xe, tuyến đường và đánh giá nhà xe thành một DataFrame duy nhất. \n
    Trả về: DataFrame chứa toàn bộ dữ liệu chuyến xe.
    '''

    dict_route = parse_route_info(soup)   # Lấy thông tin tuyến đường (đi - đến)
    containers = soup.find_all("div", class_="container")   # Tìm tất cả container chứa chuyến xe

    lst_trips_info = []

    for container in containers:
        # Lấy thông tin chuyến xe và gộp với tuyến đường
        dict_trip_info = compile_trip_info(container) | dict_route

        df_trip_info = pd.DataFrame([dict_trip_info])  # Đưa về 1 hàng dataframe

        lst_trips_info.append(df_trip_info)

    # Gộp toàn bộ chuyến xe lại thành 1 dataframe
    all_trips_info = pd.concat(lst_trips_info, ignore_index=True)

    return all_trips_info

---

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
import time, random
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

## 5. Handle Button logic

### 5.1. Button `Xem thêm chuyến`

In [ ]:
def click_load_more_until_end(driver, max_wait: int = 10):
    """
    Nhấn 'Xem thêm chuyến' liên tục cho đến khi không còn nút nào hiển thị.
    Tối ưu tốc độ: không dùng time.sleep, chỉ chờ khi cần.
    """

    wait = WebDriverWait(driver, max_wait)
    last_count = 0

    while True:
        try:
            # Chờ nút "Xem thêm chuyến" sẵn sàng để click
            load_more_button = wait.until(
                EC.element_to_be_clickable((By.CLASS_NAME, "load-more"))
            )
            driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
            load_more_button.click()

            # Chờ đến khi trang có thêm phần tử mới xuất hiện (so với lần trước)
            wait.until(
                lambda d: len(d.find_elements(By.CLASS_NAME, "trip-item")) > last_count
            )
            last_count = len(driver.find_elements(By.CLASS_NAME, "trip-item"))

        except TimeoutException:
            # Không còn nút hoặc không tải thêm chuyến mới
            break
        except (NoSuchElementException, ElementClickInterceptedException):
            break

### 5.2. Button `Xem các đánh giá`

In [ ]:
def expand_all_ratings(driver, max_clicks=30, delay_range=(0.5, 1.2)):
    """
    Mở tất cả các cửa sổ rating trên trang Vexere (nếu có).
    - Giới hạn số lượt click để tránh treo.
    - Tự bỏ qua khi element không khả dụng.
    """

    wait = WebDriverWait(driver, 10)

    try:
        # Lấy toàn bộ các icon ngôi sao hiện có
        stars = wait.until(
            EC.presence_of_all_elements_located(
                (By.CLASS_NAME, "bus-rating-button")
            )
        )
        print(f"⭐ Tìm thấy {len(stars)} icon rating.")

        clicks_performed = 0

        for star in stars:
            if clicks_performed >= max_clicks:
                print(f"⏹ Dừng lại sau {max_clicks} lần click (tránh treo).")
                break

            try:
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", star)
                time.sleep(random.uniform(*delay_range))

                # Dùng click an toàn bằng JS (ít lỗi hơn ActionChains)
                driver.execute_script("arguments[0].click();", star)
                clicks_performed += 1

                print(f"✅ Click rating {clicks_performed}/{len(stars)}")
                time.sleep(random.uniform(*delay_range))

            except Exception as e:
                print("⚠️ Bỏ qua 1 icon do lỗi:", e)
                continue

        print("🎯 Hoàn tất mở rating!")

    except Exception as e:
        print("❌ Không thể lấy danh sách rating:", e)

### 5.3. Button `Tìm kiếm`

In [341]:
def click_search(driver):  
    """
    Click vào nút tìm kiếm trên trang Vexere.

    Parameters
    ----------
    driver : webdriver
        Đối tượng Selenium WebDriver đang điều khiển trình duyệt.

    Returns
    -------
    bool
        True nếu click thành công, False nếu xảy ra lỗi.
    """
    
    try:
        button = driver.find_element(By.CLASS_NAME,"button-search")
        button.click()
        return True
    except Exception:
        return False

## 6. Automate the process of filtering website data

In [342]:
def get_target_date_components(days=0):
    """
    Trả về ngày và tháng-năm mục tiêu cách hiện tại `days` ngày.

    Parameters
    ----------
    days : int, optional
        Số ngày cộng thêm từ ngày hiện tại (mặc định = 0).

    Returns
    -------
    dict
        {'day': '15', 'month_year': '10-2025'}
    """
    target_date = datetime.today() + timedelta(days=days)  # Ngày mục tiêu = ngày hiện tại + khoảng thời gian sau k days ngày
    month_id = f"{target_date.month:02d}-{target_date.year}"
    day = str(target_date.day)
    return {
        'day':day,
        'month_year':month_id
    }

In [343]:
def set_search_filters(driver,start_city:str, destination_city:str, days=0):
    """
    Chọn điểm đi, điểm đến và ngày khởi hành trên trang Vexere.

    Parameters
    ----------
    driver : webdriver
        Đối tượng Selenium WebDriver đang điều khiển trình duyệt.
    start_city : str
        Tên thành phố khởi hành.
    destination_city : str
        Tên thành phố điểm đến.
    days : int, optional
        Số ngày tính từ hôm nay để chọn ngày đi (mặc định = 0).

    Returns
    -------
    bool
        True nếu chọn ngày thành công, False nếu xảy ra lỗi.
    """

    # Tìm ô chứa nơi khởi hành và nơi đến
    departure_input = driver.find_element(By.ID, 'from_input')
    destination_input = driver.find_element(By.ID, 'to_input')

    # Nhập dữ liệu
    departure_input.send_keys(start_city)
    destination_input.send_keys(destination_city)
    
    # click vào chọn ngày đi để hiển thị các lựa chọn
    driver.find_element(By.CLASS_NAME, "departure-date-select").click() 
    time.sleep(1)

    # Gọi hàm target_time() -> ngày, tháng mình muốn hiển thị từ trang web
    target_day, target_month = get_target_date_components(days).values() 

    # Chọn ngày trong month_section
    month_section = driver.find_element(By.ID, target_month)
    day_elements = month_section.find_elements(By.CSS_SELECTOR, "p.day")
    
    for day in day_elements:
        if day.text == target_day:
            try:
                day.click()
                break
            except Exception:
                return False
            
    return True

# FLOW OFFICIAL

---

### check data trong database

In [ ]:
# expand_all_ratings_realtime.py
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
import time


def check_and_update_bus_in_db(db_connection, bus_name, overall_rating, reviewer_count):
    """
    Kiểm tra dữ liệu nhà xe trong DB:
    - Nếu chưa có: return "new"
    - Nếu đã có và reviewer_count khác: return "update"
    - Nếu giống hệt: return "skip"
    """
    query = """
        SELECT overall_rating, reviewer_count
        FROM bus_companies
        WHERE bus_company_name = %s
        LIMIT 1;
    """
    existing = db_connection.fetch(query, (bus_name,))

    if not existing:
        return "new"

    db_overall, db_count = existing[0]
    if reviewer_count != db_count:
        return "update"
    return "skip"


def click_rating_if_needed(driver, container, action_type):
    """
    Thực hiện click để mở phần đánh giá nếu cần.
    """
    if action_type not in ("new", "update"):
        return False

    try:
        star_elem = WebDriverWait(container, 15).until(
            EC.element_to_be_clickable((By.CLASS_NAME, ".bus-rating-button"))
        )
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", star_elem)
        ActionChains(driver).move_to_element(star_elem).click().perform()
    except Exception:
        pass


def expand_all_ratings(driver, max_wait: int = 15, db_connection=None):
    """
    Mở phần đánh giá real-time:
    - Nếu nhà xe mới → click để thêm mới.
    - Nếu nhà xe cũ nhưng có thêm lượt review → click để cập nhật.
    - Nếu giống hệt → bỏ qua.
    """
    try:
        containers = WebDriverWait(driver, max_wait).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "container"))
        )
        if not containers:
            print("⚠️ Không tìm thấy chuyến xe nào.")
            return False

        clicked_any = False

        for container in containers:
            try:
                bus_name = container.find_element(By.CLASS_NAME, "bus-name").text.strip()
                rating_elem = container.find_element(By.CSS_SELECTOR, ".bus-rating span")
                rating_text = rating_elem.text.strip()
                parts = rating_text.split(" ")

                if len(parts) < 2:
                    print(f"⚠️ Rating không hợp lệ: {rating_text}")
                    continue

                overall_rating = float(parts[0])
                reviewer_count = int(parts[1].strip("()"))

                action_type = "new"
                if db_connection:
                    action_type = check_and_update_bus_in_db(
                        db_connection, bus_name, overall_rating, reviewer_count
                    )

                if action_type in ("new", "update"):
                    clicked = click_rating_if_needed(driver, container, bus_name, rating_text, action_type)
                    if clicked and action_type == "update":
                        # Cập nhật DB ngay
                        update_query = """
                            UPDATE bus_companies
                            SET overall_rating = %s, reviewer_count = %s, updated_at = NOW()
                            WHERE bus_company_name = %s;
                        """
                        db_connection.execute(update_query, (overall_rating, reviewer_count, bus_name))
                    elif clicked and action_type == "new":
                        insert_query = """
                            INSERT INTO bus_companies (bus_company_name, overall_rating, reviewer_count)
                            VALUES (%s, %s, %s);
                        """
                        db_connection.execute(insert_query, (bus_name, overall_rating, reviewer_count))
                    clicked_any = clicked
                else:
                    print(f"⏭️  Bỏ qua {bus_name} — dữ liệu không thay đổi.")

            except Exception as e:
                print(f"❌ Lỗi xử lý container: {e}")
                continue

        print("🎯 Hoàn tất kiểm tra & mở đánh giá.")
        return clicked_any

    except Exception as e:
        print("⚠️ Lỗi chung trong expand_all_ratings():", e)
        return False


## Crawl rating data

# -- Main --

In [345]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time

In [346]:
arrivals_HaNoi = ['Hải Phòng','Nghệ An','Sơn La','Hà Giang','Quảng Ninh','Thanh Hóa'',SaPa','Ninh Bình']
arrivals_SaiGon = ['Gia Lai','Bình Thuận','Ninh Thuận','Đắk Lắk','Phú Yên','Nha Trang','Bà Rịa - Vũng Tàu']

In [347]:
# URL = 'https://vexere.com/'
URL = "https://vexere.com/vi-VN/ve-xe-khach-tu-sai-gon-di-nha-trang-khanh-hoa-129t23591.html?date=17-10-2025&v=8&nation=84"
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(URL)

In [ ]:
import sys
import os
import pathlib
import json
# 1. Xác định vị trí của Notebook (ví dụ: src/crawling)
notebook_path = pathlib.Path.cwd() 

# 2. Đi lên 1 cấp để tới thư mục gốc của dự án (src)
project_root = str(notebook_path.parent) # src/

# 3. Thêm thư mục gốc vào sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

# 4. Kiểm tra xem đường dẫn 'src' đã được thêm chưa
print(f"Thư mục gốc được thêm vào sys.path: {project_root}")
print("---")

# 5. Thực hiện Import
from database.db_connection import DatabaseManager
# --------------------------------------------------

# ===== Lấy dữ liệu connect DB từ json ======
# SỬA LỖI ĐƯỜNG DẪN CONFIG.JSON TRONG NOTEBOOK
# Notebook đang ở src/crawling -> cần đi lên src/ rồi vào database/
json_path = pathlib.Path().resolve().parent.parent /'src' / 'database' / 'config.json'

with open(json_path, "r", encoding="utf-8") as f:
    config = json.load(f)

DATABASE = config["DB_CONNECTION"]
host = DATABASE["HOST"]
port = DATABASE["PORT"]
database = DATABASE["DATABASE"]
user = DATABASE["USER"]
password = DATABASE["PASSWORD"]

# ====== Khỏi tạo Database ======
db_manager = DatabaseManager(
    host=host, port=port, user=user, database=database, password=password
)

departure_place = 'Sài Gòn' # Nơi xuất phát
days_offset = 1

day, month_year = get_target_date_components(days_offset).values()
month_year = day.replace('-', '_')

for i in arrivals_SaiGon:
    arrival_city = i

    filter_success = set_search_filters(driver, start_city=departure_place, destination_city=arrival_city, days=days_offset)

    if filter_success:
        search_suess = click_search(driver)
    
    # click_load_more_until_end(driver)

    # if search_suess:
    #     expand_all_ratings(driver, 15,db_manager)

    time.sleep(1.5)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    lst_trips_info = []

    try:
        df_trips_info = extract_all_trips(soup)
    except Exception:

        with open(f'../../data/site/{departure_place}_{arrival_city}_{day}_{month_year}.html', 'w', encoding='utf-8') as f:
            f.write(soup.prettify())
        
        print("Đã lưu thành .html")

    df_trips_info.to_csv(f"../../data/raw/{departure_place}_{arrival_city}_{day}_{month_year}.csv", index=False)
    print("Lấy dữ liệu .csv thành công")

    driver.quit()
    break


Thư mục gốc được thêm vào sys.path: f:\Document\Tổng hợp các môn học\Đồ án DS\src
---
Lấy dữ liệu .csv thành công


In [349]:
df_trips_info

,bus_name,bus_rating,seat_type,departure_time,pick_up_point,arrival_date,arrival_time,drop_of_point,duration,original_price,sale_price,percent_discount,departure_date,start_point,destination
0,Đà Lạt ơi,4.8 (3452),Limousine 24 Phòng ĐÔI,23:45,• Trạm Quận 1,(18/10),05:30,• Trạm Nha Trang,5h45m,450.000,279.000,-38%,"T6, 17/10/2025",Sài Gòn,Nha Trang - Khánh Hòa
1,Trọng Thủy Limousine,4.7 (1353),Limousine 24 phòng Đôi,21:00,• Ngã 4 An Sương,(18/10),04:15,• Văn Phòng Nha Trang,7h15m,470.000,430.000,-9%,"T6, 17/10/2025",Sài Gòn,Nha Trang - Khánh Hòa
2,Khanh Phong,4.7 (16973),Limousine 32 giường nằm (WC),22:45,• Văn Phòng Phạm Ngũ Lão - Quận 1.,(18/10),04:55,• Văn Phòng Nha Trang (KS Mường Thanh),6h10m,320.000,300.000,-6%,"T6, 17/10/2025",Sài Gòn,Nha Trang - Khánh Hòa
3,Bình Minh Tải,4.7 (3701),Limousine 32 Phòng (WC),12:15,• Văn Phòng Quận 1,None,19:20,• Văn phòng Nha Trang,7h5m,320.000,280.000,-12%,"T6, 17/10/2025",Sài Gòn,Nha Trang - Khánh Hòa
4,Nhật Dương - Bình Minh Bus,4.9 (5872),Limousine 22 Phòng Đôi Luxury (WC),23:00,• Văn phòng Nguyễn Cư Trinh Quận 1,(18/10),05:36,• Vp Thích Quảng Đức Nha Trang,6h36m,349.000,300.000,-14%,"T6, 17/10/2025",Sài Gòn,Nha Trang - Khánh Hòa
5,An Anh Limousine,4.8 (8389),Limousine 34 Phòng Đơn,23:30,• Văn Phòng Quận 5,(18/10),06:00,• Văn phòng Nha Trang,6h30m,299.000,250.000,-16%,"T6, 17/10/2025",Sài Gòn,Nha Trang - Khánh Hòa
6,Huỳnh Gia,4.7 (8578),Giường nằm 34 chỗ (WC),22:30,• Văn Phòng Phạm Ngũ Lão,(18/10),05:00,• Văn Phòng Nha Trang,6h30m,280.000,None,None,"T6, 17/10/2025",Sài Gòn,Nha Trang - Khánh Hòa
7,Liên Hưng,4 (12979),Limousine 21 phòng đơn (WC),21:45,• Bến xe Miền Tây (Quầy 24),(18/10),06:30,• Văn phòng Nha Trang,8h45m,500.000,300.000,-40%,"T6, 17/10/2025",Sài Gòn,Nha Trang - Khánh Hòa
8,Nam Hải Limousine,4.7 (3509),Limousine 34 giường,22:20,• Văn Phòng Phạm Ngũ Lão,(18/10),06:20,• Văn Phòng Nha Trang,8h,300.000,270.000,-10%,"T6, 17/10/2025",Sài Gòn,Nha Trang - Khánh Hòa
9,Phương Nam,4.7 (2442),Giường nằm 40 chỗ có toilet,22:00,• Văn phòng Quận 1,(18/10),06:52,• Văn Phòng Chung cư CT2,8h52m,240.000,None,None,"T6, 17/10/2025",Sài Gòn,Nha Trang - Khánh Hòa
